# Experiment to observe "Intra-CCA fairness by Reno, Cubic and BBR at Edge and Core Scale containing unequal number of flows"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
for n in sender_nodes:
    n.upload_file('iperf-parallel-senders.sh','iperf-parallel-senders.sh')
for n in receiver_nodes:
    n.upload_file('iperf-parallel-servers.sh','iperf-parallel-servers.sh')

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port

test_duration is the time for which to send the iperf3 flows

cca1 is the first congestion control algorithm (bbr, reno or cubic); cca2 is the second congestion control algorithm (bbr, reno or cubic)

flows is the number of parallel flows to be send from each port

For sending 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

In [ ]:
# cca1="bbr"
# cca2="reno"
# delay=20
# test_duration=60
# num_servers=10
# flows=10

In [ ]:
# generate full factorial experiment

import itertools
exp_factors_core = {
    'scenario': ['core'], 
    'rate': ['10Gbit'],
    'limit': ['375MB'],
    'cca': ["bbr-reno","bbr-cubic"],
    'delay': [20,100,200],
    'test_duration': [300],
    'num_servers': [300, 500, 700],
    'flows': [1],
    'interval': [0.01],
    'omit': [0],
    'trial': [1]
}

factor_names = [k for k in exp_factors_core]
factor_lists = list(itertools.product(*exp_factors_core.values()))
exp_lists_core = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

exp_factors_edge = { 
    'scenario': ['edge'], 
    'rate': ['100Mbit'],
    'limit': ['3MB'],
    'cca': ["bbr-reno","bbr-cubic"],
    'delay': [20,100,200],
    'test_duration': [300],
    'num_servers': [3, 5,7],
    'flows': [1],
    'interval': [0.01],
    'omit': [0],
    'trial': [1]
}
factor_names = [k for k in exp_factors_edge]
factor_lists = list(itertools.product(*exp_factors_edge.values()))
exp_lists_edge = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

exp_factors_inter = { 
    'scenario': ['inter'], 
    'rate': ['1Gbit'],
    'limit': ['25MB'],
    'cca': ["bbr-reno","bbr-cubic"],
    'delay': [20,100,200],
    'test_duration': [1800],
    'num_servers': [30, 50, 70],
    'flows': [1],
    'interval': [0.01],
    'omit': [0],
    'trial': [1]
}

factor_names = [k for k in exp_factors_edge]
factor_lists = list(itertools.product(*exp_factors_inter.values()))
exp_lists_inter = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]


exp_lists = exp_lists_core + exp_lists_edge

# Start the experiment 

In [ ]:
import os
directory_name = "intercca-unequal"
if not os.path.exists(directory_name):
    # Create the directory
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
else:
    print(f"Directory '{directory_name}' already exists.")
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

In [ ]:
# Specify the base port to start the connections 
base_port=50000

In [ ]:
import time
for exp in exp_lists: 
    # check if we already ran this experiment
    # (allow stop/resume)
    exp_name_str = "_".join( [str(v) for v in exp.values()] )
    iunequal_file_out = data_dir + '/unequal_' + "_".join( [str(v) for v in exp.values()] )+ ".csv" # file with throughput saved

    # TODO check if the c file and j file already exist
    if (os.path.exists(iunequal_file_out)):
        print("Already have " + iunequal_file_out + ", skipping")

    else:
        print("Running experiment to generate " +  iunequal_file_out)
        
        # set up edge or core scale setting
        # first delete any existing queue
        router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")
        # then set one up with HW offload
        router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3 offload")
        router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + exp['rate'])
        router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 handle 3: bfifo limit " + exp['limit'])

        ## Remove existing result files from the hosts #Check if the files are removed from the senders and receivers
        for n in receiver_nodes:
            #n.execute("rm -f 60*")
            #n.execute("rm -f ./*")
            n.execute("rm -f server*")

        for n in sender_nodes:
            #n.execute("rm -f ./*")
            #n.execute("rm -f 60*")
            n.execute("rm -f sender*")
            #n.execute("rm -f data*")
            #n.execute("rm -f packet*")
            #n.execute("rm -f output*")

        #Now set up delay on the receiver interface:
        #First delete any existing queue (don't worry if there is an error, it means there was not!)
        for n in receiver_nodes:
            receiver_inf=n.get_interface(network_name= "link-receiver")
            receiver_inf_name = receiver_inf.get_device_name()
            n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
            n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(exp['delay']) +"ms limit 1000000")

        ### Start parallel servers on the receivers
        #In this, the base_port is the starting address of port number
        
        print("Start parallel servers on the receivers")
        for i, n in enumerate(receiver_nodes):
            n.execute("sudo killall iperf3")
            n.execute_thread(f'chmod +x iperf-parallel-servers.sh && bash iperf-parallel-servers.sh '+str(exp['num_servers']+1)+" "+str(base_port))
        

        ### Start parallel clients on the senders
        #base_port=60000
        
        print("Start parallel clients on the senders")
        for i,n in enumerate(sender_nodes):
            n.execute("sudo killall iperf3")
            n.execute_thread(f'chmod +x iperf-parallel-senders.sh && bash iperf-parallel-senders.sh 10.10.2.1'+str(i)+" "+str(exp['num_servers'])+" "+str(exp['test_duration'])+" "+exp['cca'].split("-")[1]+" "+str(exp['flows'])+" "+str(exp['interval'])+" "+str(exp['omit'])+" "+str(base_port))
            
            if i==9:
                n.execute(f'chmod +x iperf-parallel-senders.sh && bash iperf-parallel-senders.sh 10.10.2.1'+str(i)+" 1 "+str(exp['test_duration'])+" "+exp['cca'].split("-")[0]+" 1 "+str(exp['interval'])+" "+str(exp['omit'])+" "+str(base_port+exp['num_servers']))

        time.sleep(exp['test_duration']+180)

        # Code for files to be saved/copied inside the sender 
        exp_dir = "intercca-unequal/" + "_".join( [str(v) for v in exp.values()])
        for i,n in enumerate(sender_nodes):    
            n.execute(f'mkdir -p {exp_dir} && cp sender* {exp_dir}/')
        

        ## Analyze the results
        # Transfer files from hosts to router
        # Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

        sum_bw1 = []
        count_flow1 = []
        sum_bw2 = []
        count_flow2 = []

        for i,n in enumerate(sender_nodes):
            (sum_sen1, serr1)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include \"*"+str(exp['test_duration'])+"-"+exp['cca'].split("-")[1]+".txt\" --exclude-dir=\"*int*\"  . |awk '{sum+=$7}END {print sum}'")  
            sum_bw1.append(float(sum_sen1.strip()))
            (ncount1,ncerr1)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include \*"+exp['cca'].split("-")[1]+".txt --exclude-dir=\"*int*\" . |awk '{count+=1}END {print count}'")
            count_flow1.append(int(ncount1.strip()))
            if i==9:
                (sum_sen2, serr2)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include \"*"+str(exp['test_duration'])+"-"+exp['cca'].split("-")[1]+".txt\" --exclude-dir=\"*int*\"  . |awk '{sum+=$7}END {print sum}'")  
                sum_bw2.append(float(sum_sen2.strip()))
                (ncount2,ncerr2)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include \*"+exp['cca'].split("-")[0]+".txt --exclude-dir=\"*int*\" . |awk '{count+=1}END {print count}'")
                count_flow2.append(int(ncount2.strip()))

        tput1=sum(sum_bw1)
        c1=sum(count_flow1)
        tput2=sum(sum_bw2)
        c2=sum(count_flow2)
        tp_share = (tput2*100)/(tput1 + tput2)

        print("Experiment name " + "_".join( [str(v) for v in exp.values()]))
        print("Sum of bandwidth of "+exp['cca'].split("-")[0]+ " is %f Kbits/sec " % tput2)
        print("Count of flows of " +exp['cca'].split("-")[0]+ " is " + str(c2))
        print("Sum of bandwidth of "+exp['cca'].split("-")[1]+ " is %f Kbits/sec " % tput1)
        print("Count of flows of " +exp['cca'].split("-")[1]+ " is " +  str(c1))
        print(str(c2) + " flows of " + exp['cca'].split("-")[0] +  " has a throughput share of " + str(tp_share) + " against " + str(c1)+  " flows of " + exp['cca'].split("-")[1])       


        if not os.path.isfile(iunequal_file_out):
            with open(iunequal_file_out, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                header ='cca1', 'Number of flows of cca1', 'cca2', 'Number of flows of cca2', 'rtt', 'throughput_percentage_share'
                writer.writerow(header)

        with open(iunequal_file_out, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            columns = exp['cca'].split("-")[0], c2, exp['cca'].split("-")[1], c1, exp['delay'], tp_share
            writer.writerow(columns)
        

## Fig 6 Plot

In [ ]:
## Plots 

# Fig 6
# Core scale 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

N = 3
ind = np.arange(N) 
width = 0.25
color_name = {'20':'royalblue', '100':'tomato', '200':'mediumspringgreen'}
xvals = [0,0,0]
bar = [0,0,0]

directory_name = "intercca-unequal"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

# List of filenames for core and edge 
iunequal_files_core = [data_dir + '/unequal_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core] # file with JFI
iunequal_files_edge = [data_dir + '/unequal_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge] # file with JFI


#  First let us plot for Core 
#Read each CSV file and append its DataFrame to the list and concatenate it later
dfs = []
for filename in iunequal_files_core:
    df  = pd.read_csv(filename, header=0, names=['cca1', 'cca2', 'rtt', 'number_of_flows', 'throughput_percentage'])
    dfs.append(df)  
iunequal_dat_core = pd.concat(dfs, ignore_index=True) 


## Figure 6
with PdfPages("Fig6_Core.pdf") as pdf:  
    plt.rcParams['figure.figsize'] = (5,3)
    plt.rcParams['axes.axisbelow'] = True
    plt.grid(axis='y')
    dat_cca=iunequal_dat_core[(iunequal_dat_core['cca1'] == 'bbr') and (iunequal_dat_core['cca2'] == 'reno')]
    dat_cca.sort_values(by=['number_of_flows'])
    #print(dat_cca)
    flows=pd.unique(dat_cca.number_of_flows)
    rtt_num = pd.unique(dat_cca.rtt)       
    for j,r in enumerate(rtt_num):
        xvals[j] = dat_cca.throughput_percentage[dat_cca['rtt']==r]
        bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])

    plt.xlabel("Flow Count")
    plt.ylabel('BBR Throughput(%)')
    plt.title("Core Scale : BBR vs NewReno Unequal flows')        
    plt.xticks(ind+width,flows)
    plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
    #pdf.savefig(bbox_inches="tight")
    plt.show()
    plt.close()
        
#  Now let us plot for Edge 
#Read each CSV file and append its DataFrame to the list and concatenate it later

dfs = []
for filename in iunequal_files_edge:
    df  = pd.read_csv(filename, header=0, names=['cca1', 'cca2', 'rtt', 'number_of_flows', 'throughput_percentage'])
    dfs.append(df)  
iunequal_dat_edge = pd.concat(dfs, ignore_index=True) 

with PdfPages("Fig6_Edge.pdf") as pdf:  
    plt.rcParams['figure.figsize'] = (5,3)
    plt.rcParams['axes.axisbelow'] = True
    plt.grid(axis='y')
    dat_cca=iunequal_dat_edge[(iunequal_dat_edge['cca1'] == 'bbr') and (iunequal_dat_edge['cca2'] == 'reno')]
    dat_cca.sort_values(by=['number_of_flows'])
    #print(dat_cca)
    flows=pd.unique(dat_cca.number_of_flows)
    rtt_num = pd.unique(dat_cca.rtt)       
    for j,r in enumerate(rtt_num):
        xvals[j] = dat_cca.throughput_percentage[dat_cca['rtt']==r]
        bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])

    plt.xlabel("Flow Count")
    plt.ylabel('BBR Throughput(%)')
    plt.title("Edge Scale : BBR vs NewReno Unequal flows')        
    plt.xticks(ind+width,flows)
    plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
    #pdf.savefig(bbox_inches="tight")
    plt.show()
    plt.close()




## Fig 6

In [ ]:
## Plots 

# Fig 6
# Core scale 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

N = 3
ind = np.arange(N) 
width = 0.25
color_name = {'20':'royalblue', '100':'tomato', '200':'mediumspringgreen'}
xvals = [0,0,0]
bar = [0,0,0]

directory_name = "intercca-unequal"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

# List of filenames for core and edge 
iunequal_files_core = [data_dir + '/unequal_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core] # file with JFI
iunequal_files_edge = [data_dir + '/unequal_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge] # file with JFI


#  First let us plot for Core 
#Read each CSV file and append its DataFrame to the list and concatenate it later
dfs = []
for filename in iunequal_files_core:
    df  = pd.read_csv(filename, header=0, names=['cca1', 'cca2', 'rtt', 'number_of_flows', 'throughput_percentage'])
    dfs.append(df)  
iunequal_dat_core = pd.concat(dfs, ignore_index=True) 


## Figure 5
with PdfPages("Fig7_Core.pdf") as pdf:  
    plt.rcParams['figure.figsize'] = (5,3)
    plt.rcParams['axes.axisbelow'] = True
    plt.grid(axis='y')
    dat_cca=iunequal_dat_core[(iunequal_dat_core['cca1'] == 'bbr') and (iunequal_dat_core['cca2'] == 'cubic')]
    dat_cca.sort_values(by=['number_of_flows'])
    #print(dat_cca)
    flows=pd.unique(dat_cca.number_of_flows)
    rtt_num = pd.unique(dat_cca.rtt)       
    for j,r in enumerate(rtt_num):
        xvals[j] = dat_cca.throughput_percentage[dat_cca['rtt']==r]
        bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])

    plt.xlabel("Flow Count")
    plt.ylabel('BBR Throughput(%)')
    plt.title("Core Scale : BBR vs Cubic Unequal flows')        
    plt.xticks(ind+width,flows)
    plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
    #pdf.savefig(bbox_inches="tight")
    plt.show()
    plt.close()
        
#  Now let us plot for Edge 
#Read each CSV file and append its DataFrame to the list and concatenate it later

dfs = []
for filename in iunequal_files_edge:
    df  = pd.read_csv(filename, header=0, names=['cca1', 'cca2', 'rtt', 'number_of_flows', 'throughput_percentage'])
    dfs.append(df)  
iunequal_dat_edge = pd.concat(dfs, ignore_index=True) 

with PdfPages("Fig7_Edge.pdf") as pdf:  
    plt.rcParams['figure.figsize'] = (5,3)
    plt.rcParams['axes.axisbelow'] = True
    plt.grid(axis='y')
    dat_cca=iunequal_dat_edge[(iunequal_dat_edge['cca1'] == 'bbr') and (iunequal_dat_edge['cca2'] == 'cubic')]
    dat_cca.sort_values(by=['number_of_flows'])
    #print(dat_cca)
    flows=pd.unique(dat_cca.number_of_flows)
    rtt_num = pd.unique(dat_cca.rtt)       
    for j,r in enumerate(rtt_num):
        xvals[j] = dat_cca.throughput_percentage[dat_cca['rtt']==r]
        bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])

    plt.xlabel("Flow Count")
    plt.ylabel('BBR Throughput(%)')
    plt.title("Edge Scale : BBR vs Cubic Unequal flows')        
    plt.xticks(ind+width,flows)
    plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
    #pdf.savefig(bbox_inches="tight")
    plt.show()
    plt.close()




In [ ]:
# import csv
# import sys
# import os

# jfi_filename='jfi.csv'
# if not os.path.isfile(jfi_filename):
#     with open(jfi_filename, 'a', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         # header ='CCA1', 'CCA2', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'BW_CCA1', 'BW_CCA2', 'Count_CCA1', 'Count_CCA2', 'BW_CCA1/BW'
#         writer.writerow(header)
    
# with open(jfi_filename, 'a', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     columns = cca1, cca2, test_duration, delay, tput1+tput2, tput1, tput2, c1, c2, tput2/(tput1+tput2)
#     writer.writerow(columns)
